In [38]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd

# Always make it pretty.
plt.style.use('ggplot')

In [43]:
df1 = pd.read_csv("data/accidents_2005_to_2007.csv")
df2 = pd.read_csv("data/accidents_2009_to_2011.csv")
df3 = pd.read_csv("data/accidents_2012_to_2014.csv")
traffic = pd.read_csv("data/ukTrafficAADF.csv")

df12 = pd.concat([df1,df2])
df = pd.concat([df12,df3])

df = df.drop_duplicates(subset=['Accident_Index','Date','LSOA_of_Accident_Location','Time','Longitude','Latitude'], keep='first')

df = df[df.Speed_limit !=15]
# df = df[(df.Weather_Conditions == 'Fine without high winds') | (df.Weather_Conditions =='Raining without high winds')]
df = df[(df.Road_Surface_Conditions != 'Flood (Over 3cm of water)') & (df.Road_Surface_Conditions !='Snow')]

In [44]:
del df['Police_Force']
del df['Local_Authority_(Highway)']
del df['Local_Authority_(District)']
del df['Pedestrian_Crossing-Human_Control']
del df['Pedestrian_Crossing-Physical_Facilities']
del df['Did_Police_Officer_Attend_Scene_of_Accident']

In [45]:
severity_totals = df.groupby('Accident_Severity').size().values
#total is 1,469,963 number of accidents 1469963
severity_mus = severity_totals / severity_totals.sum()
severity_mus

array([0.01298486, 0.13558424, 0.8514309 ])

In [61]:
#actuals number of accidents of severity 0,1,2, (columns) and Weather conditions sunny (0) and rainy (1) rows
# actuals_severity_roadsc = df.groupby(['Accident_Severity','Road_Surface_Conditions']).count()['Accident_Index']

actuals_severity_roadsc = np.transpose(np.reshape(df.groupby(['Accident_Severity','Road_Surface_Conditions']).count()['Accident_Index'].values,(3,3)))
actuals_severity_roadsc

array([[ 12751, 139816, 858929],
       [   317,   3523,  26809],
       [  5846,  54081, 353527]])

In [62]:
sums2 = df.groupby(['Accident_Severity','Road_Surface_Conditions']).count().groupby('Accident_Severity').sum()['Accident_Index']
severity_sums = df.groupby(['Accident_Severity','Road_Surface_Conditions']).count().groupby('Accident_Severity').sum()['Accident_Index'].values
severity_mus = severity_sums/severity_sums.sum()
severity_mus

array([0.01299396, 0.13562801, 0.85137802])

In [63]:
sums = df.groupby(['Road_Surface_Conditions']).count()['Accident_Index']
roadsc_sums = df.groupby(['Road_Surface_Conditions']).count()['Accident_Index'].values
roadsc_sums

array([1011496,   30649,  413454])

In [64]:
expected_by_roadsc = np.zeros((3,3))
for i in np.arange(3):
    for j in np.arange(3):
        expected_by_roadsc[i][j] = roadsc_sums[i] * severity_mus[j]
expected_by_roadsc

array([[1.31433419e+04, 1.37187193e+05, 8.61165466e+05],
       [3.98251981e+02, 4.15686297e+03, 2.60938851e+04],
       [5.37240611e+03, 5.60759445e+04, 3.52005649e+05]])

In [65]:
expected_by_roadsc = expected_by_roadsc.round().astype(int)
expected_by_roadsc
expected_by_roadsc.sum(axis=1)

array([1011495,   30649,  413454])

In [66]:
pvalues = np.zeros(3)

# stats.chisquare(actuals_severity_slimits[0],expected_by_slimit[0])[1]

for i in np.arange(2):
    pvalues[i] = stats.chisquare(actuals_severity_roadsc[i],expected_by_roadsc[i])[1]

pvalues

array([1.82116488e-15, 1.47666523e-29, 0.00000000e+00])